In [9]:
from nltk.corpus import stopwords

In [2]:
a="On 20 August 1945, the Indonesian government established the Agency for Assisting Families of War Victims (Badan Penolong Keluarga Korban Perang; abbreviated as BPKKP) and its focus was to assist the war victims and their respective family during the time of Indonesian National Revolution.In 1966, the government established the Advisory Board of Central Natural Disaster Management (Badan Pertimbangan Penanggulangan Bencana Alam Pusat; abbreviated as BP2BA) through the Presidential Decree Number 256 of 1966. The board was responsible to the Minister of Social Affairs. A year later, the Cabinet Presidium through Decree number 14/U/KEP/I/1967, established the National Coordination Team for Disaster Management (Tim Koordinasi Nasional Penanggulangan Bencana Alam; abbreviated as TKP2BA).Bakornas PB was established in 1979 to replace the Advisory Board for Natural Disaster, which was established in 1966."

In [24]:
import re
import operator

def is_number(s):
    try:
        float(s) if '.' in s else int(s)
        return True
    except ValueError:
        return False
def load_stop_words():
    stop = set(stopwords.words('english'))
    stop_words = []
    for i in stop:
        stop_words.append(i)
    return stop_words
def build_stop_word_regex():
    stop_word_list = load_stop_words()
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = r'\b' + word + r'(?![\w-])'
        stop_word_regex_list.append(word_regex)
    stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)
    return stop_word_pattern


def split_sentences(text):
    """
    Utility function to return a list of sentences.
    @param text The text that must be split in to sentences.
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    return sentences




def separate_words(text):
    """9
    Utility function to return a list of all words that are have a length greater than a specified number of characters.
    @param text The text that must be split in to words.
    @param min_word_return_size The minimum no of characters a word must have to be included.
    """
    splitter = re.compile('\W+')
    words = []
    for single_word in splitter.split(text):
        current_word = single_word.strip().lower()
        # leave numbers in phrase, but don't count as words, since they tend to invalidate scores of their phrases
        if current_word != '' and not is_number(current_word):
            words.append(current_word)
    return words




def generate_candidate_keywords(sentence_list, stop_word_pattern, minCharacters, maxWords):
    phrase_list = []
    for s in sentence_list:
        tmp = re.sub(stop_word_pattern, '|', s.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            if phrase != '' and len(phrase) >= minCharacters and len(phrase.split()) <= maxWords:
                phrase_list.append(phrase)
    return phrase_list


def calculate_word_scores(phraseList):
    word_frequency = {}
    word_degree = {}
    for phrase in phraseList:
        word_list = separate_words(phrase)
        word_list_length = len(word_list)
        word_list_degree = word_list_length - 1
        for word in word_list:
            word_frequency.setdefault(word, 0)
            word_frequency[word] += 1
            word_degree.setdefault(word, 0)
            word_degree[word] += word_list_degree
    for item in word_frequency:
        word_degree[item] = word_degree[item] + word_frequency[item]

    # Calculate Word scores = deg(w)/frew(w)
    word_score = {}
    for item in word_frequency:
        word_score.setdefault(item, 0)
        word_score[item] = word_degree[item] / (word_frequency[item] * 1.0)
    return word_score


def generate_candidate_keyword_scores(phrase_list, word_score, minFrequency):
    keyword_candidates = {}
    for phrase in phrase_list:
        if phrase_list.count(phrase) >= minFrequency:
            keyword_candidates.setdefault(phrase, 0)
            word_list = separate_words(phrase)
            candidate_score = 0
            for word in word_list:
                candidate_score += word_score[word]
            keyword_candidates[phrase] = candidate_score
    return keyword_candidates


class Rake(object):
    def __init__(self,regex='[\W\n]+', minCharacters=1, maxWords=5, minFrequency=1):
        #lets users call predefined stopwords easily in a platform agnostic manner or use their own list
        if isinstance(stop_words, list):
            self.__stop_words_pattern = build_stop_word_regex()
        else:
            self.__stop_words_pattern = build_stop_word_regex()

        self.__minCharacters = minCharacters
        self.__maxWords = maxWords
        self.__minFrequency = minFrequency

    def run(self, text):
        sentence_list = split_sentences(text)

        phrase_list = generate_candidate_keywords(sentence_list, self.__stop_words_pattern, self.__minCharacters, self.__maxWords)

        word_scores = calculate_word_scores(phrase_list)

        keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores, self.__minFrequency)

        sorted_keywords = sorted(keyword_candidates.items(), key=operator.itemgetter(1), reverse=True)
        return sorted_keywords

In [25]:
rake_object =Rake(1, 1, 1)
keywords = rake_object.run(a)
print (keywords)

NameError: name 'stop_words' is not defined